# Comparing Minimum Risk Training and REINFORCE
## Diego Molla-Aliod
LTG, 27 May 2019

# Minimum Risk Training (Shen et al, 2015)
Shen, S., Cheng, Y., He, Z., He, W., Wu, H., Sun, M., & Liu, Y. (2015). Minimum Risk Training for Neural Machine Translation. Retrieved from http://arxiv.org/abs/1512.02433

MRT has the following advantages over MLE:
1. **Direct optimization with respect to evaluation metrics**: MRT introduces evaluation metrics as loss functions and aims to minimize expected loss on the training data.
2. **Applicable to arbitrary loss functions**, which are not necessarily differentiable.
3. **Transparent to architectures**: MRT does not assume the specific architectures of NMT and can be applied to ant end-to-end NMT systems.

# End-to-end NMT: Using MLE


![NMT-background](images/NMT-background.png)

![NMT-background2](images/NMT-background2.png)

![NMT-background3](images/NMT-background3.png)

# Problems with MLE

1. **Exposure bias**: While the models are trained only on the training data distribution, they are used to generate target words on previous model predictions, which can be erroneous, at test time.
2. **Loss function**: MLE usually uses the cross-entropy loss focusing on word-level errors to maximize the probability of the next correct word, which might hardly correlate well with corpus-level and sentence-level evaluation metrics such as BLEU.

# Enters MRT

In MRT, the *risk* is defined as the expected loss with respect to the posterior distribution:

$$
\begin{align}
{\cal R}(\theta) & = \sum_{s=1}^S \mathbb{E}_{y|x^{(s)};\theta}\left[ \Delta(y,y^{(s)})\right]\\
& = \sum_{s=1}^S \sum_{y\in {\cal Y}(x^{(s)})}P(y|x^{(s)};\theta) \Delta(y,y^{(s)})\\
\end{align}
$$
where ${\cal Y}(x^{(s)})$ is a set of all possible candidate translations for $x^{(s)}$, and $\Delta(y,y^{(s)})$ is a loss function. **The loss function is not parameterized and thus it does not need to be differentiable.**


The training objective of MRT is to minimise the risk on the training data:

$$\hat{\theta}_{\hbox{MRT}} = \arg\min_\theta\left\{{\cal R}(\theta)\right\}$$


In MRT, the partial derivative with respect to a model parameter $\theta_i$ is given by:
![MRT-derivative](images/MRT-derivative.png)
* Since Eq. (10) suggests there is no need to differentiate $\Delta(y,y^{(s)})$, MRT allows arbitrary non-differentiable loss functions.
* In addition, the approach is transparent to architectures and can be applied to arbitrary end-to-end NMT models.

# Solving MRT
However, $\mathbb{E}_{y|x^{(s)};\theta}\left[ \Delta(y,y^{(s)})\right]$ is usually intractable to calculate due to:
1. the exponential search space of $\cal Y(x^{(s)})$,
2. the non-decomposability of the loss function $\Delta(y,y^{(s)})$, and
3. the context sensitiveness of NMT.

To alleviate this problem, Shen et al. (2015) propose to **use a subset of the full search space to approximate the posterior distribution** and introduce a new training objective:

$$
\begin{align}
\tilde{\cal R}(\theta) & = \sum_{s=1}^S \mathbb{E}_{y|x^{(s)};\theta,\alpha}\left[ \Delta(y,y^{(s)})\right]\\
& = \sum_{s=1}^S \sum_{y\in S(x^{(s)})}Q(y\mid x^{(s)};\theta,\alpha) \Delta(y,y^{(s)})\\
Q(y\mid x^{(s)};\theta,\alpha) & = \frac{P(y\mid x^{(s)};\theta)^\alpha}{\sum_{y'\in S(x^{(s)})}P(y'\mid x^{(s)};\theta)^\alpha}\\
\end{align}
$$

Where:
1. $S(x^{(s)}) \subset \cal Y(x^{(s)})$ is a sampled subset of the search space (see Algorithm 1),
2. $Q(y\mid x^{(s)};\theta,\alpha)$ is a distribution defined on the subspace $S(x^{(s)})$, and
3. $\alpha$ is a hyper-parameter that controls the sharpness of the $Q$ distribution.

![Sampling](images/Sampling.png)

Given the sampled space, the partial derivative with respect to a model parameter $\theta_i$ of $\tilde{\cal R}(\theta)$ is given by
![MRT-derivative2](images/MRT-derivative2.png)
Since $|S(x^{(s)})|$ can be relatively small (in their experiments, 100 was good enough), the expectations in Eq.(14) can be efficiently calculated by explicitly enumerating all candidates in $S(x^{(s)})$.

But also:
$$
\begin{align}
\frac{\partial \tilde{\cal R}(\theta)}{\partial\theta_i} & = \frac{\partial}{\partial\theta_i}\left[\sum_{s=1}^S\sum_{y\in S(x^{(s)})}Q(y\mid x^{(s)};\theta,\alpha)\Delta(y,y^{(s)})\right]\\
& = \sum_{s=1}^S\sum_{y\in S(x^{(s)})}\frac{\partial Q(y\mid x^{(s)};\theta,\alpha)}{\partial\theta_i}\Delta(y,y^{(s)})\\
Q(y|x^{(s)};\theta,\alpha) & = \frac{P(y\mid x^{(s)};\theta)^\alpha}{\sum_{y'\in S(x^{(s)})}P(y'\mid x^{(s)};\theta)^\alpha}\\
\end{align}
$$


The MRT algorithm using **gradient descent** (we want to **minimise** the risk) is, therefore:

1. Sample the search space.
2. Compute $\Delta(y,y^{(s)})$ and $\frac{\partial Q(y\mid x^{(s)};\theta,\alpha)}{\partial\theta_i}$ in the sample.
3. $\theta_i \leftarrow \theta_i - \eta\frac{\partial \tilde{\cal R}(\theta)}{\partial\theta_i}$

Where $\eta$ is the learning rate.

# REINFORCE

- REINFORCE is a policy gradient approach to reinforcement learning.
- aka Monte Carlo policy differentiation.
- The agent takes an action given a state so that the expected reward $r(s,a)$ is maximum.
- A **stochastic policy** $\pi_\theta(a|s)$ is a function that has parameters $\theta$ and determines the probability of the action $a$ to take given the state $s$.
![reinforcement-learning](images/Reinforcement_learning_diagram.png)

The following material is based on:
- https://mcneela.github.io/math/2018/04/18/A-Tutorial-on-the-REINFORCE-Algorithm.html

(with slightly different notation)


The objective function $J$ returns the expected reward achieved by a given policy $\pi_{\theta}$ over some time horizon governed by $t$ (which can be either finite or infinite):
$$J(\theta) = \mathbb{E}_{\tau \sim p_\theta(\tau)} \left[ \sum_t r(s_t, a_t) \right]$$
Where $\tau \sim p_\theta(\tau)$ indicates that we're sampling trajectories $\tau$ from the probability distribution of our policy approximator governed by $\theta$.
$$p_\theta(\tau) = p_\theta(s_1, a_1, \ldots, s_T, a_T) = p(s_1) \prod_{t=1}^T \pi_\theta(a_t \mid s_t) p(s_{t+1} \mid s_t, a_t)$$
We can now specify the optimal policy $\pi^* = \pi_{\theta^*} =
arg\,max_\theta J(\theta)$.

The **Monte-Carlo Approximation** is based on the following fact:
$$\lim_{N \to \infty} \frac{1}{N}\sum_{n=1}^N f(x_n)_{x_n \sim p(x)} = \mathbb{E}[f(x)]$$
Thus, we rewrite our objective function as:
$$J(\theta) \approx \frac{1}{N} \sum_{n=1}^N \sum_{t} r(s_{n, t}, a_{n, t})$$
Where the $N$ samples are being directly drawn from the probability distribution defined by $\pi_\theta$ simply by running $\pi_\theta$, $N$ times.

It can be shown that the gradient of the loss can be computed as:
$$
\begin{align}
\frac{\partial J(\theta)}{\partial\theta_i} & = \mathbb{E}_{\tau \sim \pi_\theta(\tau)}
\left[ \left(\sum_{t} \frac{\partial\log{\pi_\theta}(a_t \mid s_t)}{\partial\theta_i}\right) \left(\sum_t r(s_t, a_t)\right)\right]\\
& \approx \frac{1}{N} \sum_{n=1}^N \left[ \left(\sum_{t} \frac{\partial\log{\pi_\theta}(a_{n, t} \mid s_{n,t})}{\partial\theta_i}\right) \left(\sum_t r(s_{n,t}, a_{n,t})\right)\right]
\end{align}
$$


The REINFORCE algorithm uses **gradient ascent** (we want to **maximise** the expected reward) to learn the parameters of the loss:
1. Sample trajectories $\left\{\tau_n\right\}_{n=1}^N$ from $\pi_\theta(a_t|s_t)$ by running the policy.
2. Set $\frac{\partial J(\theta)}{\partial\theta_i} = \sum_n \left[\left(\sum_t \frac{\partial \log \pi_\theta(a^n_t \mid s^n_t)}{\partial\theta_i}\right) \left(\sum_t r(s^n_t, a^n_t)\right)\right]$
3. $\theta \leftarrow \theta + \eta \frac{\partial J(\theta)}{\partial\theta_i}$

Within a setting for text processing (e.g. NMT, neural summarisation):

1. The action $a_t$ at step $t$ in a trajectory is the word $y_t$ generated.
    * Thus, the policy is $\pi_\theta(a_t\mid s_t) = P(y_t\mid x, y_{\lt t};\theta)$
    * MRT and REINFORCE sample from the same probability distribution $P(y_t\mid x, y_{\lt t};\theta)$
2. The trajectory is the sequence of words generated.
3. The reward of all words generated in a trajectory is 0, except the last word which has as reward the evaluation metric of the generated sentence.
    * $\sum_t r(s_t^n, y_t^n) = \Delta(y, y^{(n)})$ 
4. We have a training data which is used to train a global policy

The REINFORCE algorithm to train a global policy:
1. For each training input $x^{(s)}$, sample trajectories $\left\{\tau_n^{(s)}\right\}_{n=1}^N$ from $P(y_t^{(s)}\mid x^{(s)}, y_{\lt t}^{(s)};\theta)$ by running the policy.
2. Set $\frac{\partial J(\theta)}{\partial\theta_i} = \sum_{s=1}^S\sum_n \left[\left(\sum_t \frac{\partial \log P(y_t^{(s)}\mid x^{(s)}, y_{\lt t}^{(s)};\theta)}{\partial\theta_i}\right) \Delta(y, y^{(n)})\right]$
3. $\theta \leftarrow \theta + \eta \frac{\partial J(\theta)}{\partial\theta_i}$

* We can see that MRT and REINFORCE are very much the same.
* The functions to be differentiated in each case are very much related.
    1. MRT: $\sum_{s=1}^S\sum_{y\in S(x^{(s)})}\frac{\partial}{\partial\theta_i} \frac{P(y\mid x^{(s)};\theta)^\alpha}{\sum_{y'\in S(x^{(s)})}P(y'\mid x^{(s)};\theta)^\alpha}\Delta(y,y^{(s)})$
    2. REINFORCE: $\sum_{s=1}^S\sum_n\left[\left(\sum_t\frac{\partial}{\partial\theta_i}\log P(y_t\mid x, y_{\lt t};\theta)\right)\Delta(y,y^{(n)})\right]$
* Note that
    * if $\alpha=1$: $\frac{P(y\mid x^{(s)};\theta)^\alpha}{\sum_{y'\in S(x^{(s)})}P(y'\mid x^{(s)};\theta)^\alpha} = P(y\mid x^{(s)};\theta)$
    * $\frac{\partial}{\partial\theta_i}\log P(y\mid x^{(s)};\theta) = \sum_t\frac{\partial}{\partial\theta_i}\log P(y_t\mid x, y_{\lt t};\theta)$
* This may have implications on improving systems that use MRT, in NMT (and neural summarisation systems). 
* In particular, consider using other Policy gradient methods to calculate MRT, such as Actor-Critic, DPG, etc.